In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
from multiprocessing import reduction
import warnings
from collections import OrderedDict
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor
from tqdm import tqdm
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from numpy import vstack
from sklearn.preprocessing import StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
data_path= '/content/drive/My Drive/INTERN_2022/DRUG/data/sensitivity_data/new_aug2/' 

In [6]:
CCLE=pd.read_csv(data_path+'CCLE_paired_common_with_gcsi.csv', index_col=0)
gCSI=pd.read_csv(data_path+'gCSI_paired_common_with_ccle.csv', index_col=0)

In [7]:
ccle_x = CCLE.iloc[:, 0:-1]
ccle_y = CCLE.iloc[:, -1]

In [8]:
gCSI = gCSI.reset_index()
gCSI = gCSI.drop(columns='index')

In [9]:
gCSI_x = gCSI.iloc[:, 0:-1]
gCSI_y = gCSI.iloc[:, -1]

In [11]:
gCSI_x.iloc[0:2, 0:-20]

,711,188,733,208,130,167,881,163,878,425,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,3.700266,5.874284,5.131369,6.823007,4.542067,5.055663,3.781692,6.743258,6.557377,-3.484916,...,13.27105,11.715869,-2.953344,-8.512375,-3.749973,-6.475866,-6.030029,-12.977965,-0.488924,0
1,2.500122,5.369221,5.603282,7.696091,4.821637,4.876002,4.059001,6.677683,6.400233,-5.159899,...,13.27105,11.715869,-2.953344,-8.512375,-3.749973,-6.475866,-6.030029,-12.977965,-0.488924,0


In [12]:
gCSI_x.iloc[:, 0:-21] = StandardScaler().fit_transform(gCSI_x.iloc[:, 0:-21])
ccle_x.iloc[:, 0:-21] = StandardScaler().fit_transform(ccle_x.iloc[:, 0:-21])


In [13]:
from sklearn.model_selection import train_test_split

gCSI_X_train, gCSI_X_test, gCSI_y_train, gCSI_y_test = train_test_split(gCSI_x, gCSI_y, test_size = 0.30, random_state = 42) 
ccle_X_train, ccle_X_test, ccle_y_train, ccle_y_test = train_test_split(ccle_x, ccle_y,test_size = 0.30, random_state = 42)

In [14]:
gCSI_train = pd.concat([gCSI_X_train, gCSI_y_train], axis = 1)

In [15]:
gCSI_test = pd.concat([gCSI_X_test, gCSI_y_test], axis = 1)

In [16]:
ccle_test = pd.concat([ccle_X_test, ccle_y_test], axis = 1)

In [17]:
ccle_train = pd.concat([ccle_X_train, ccle_y_train], axis = 1)

In [18]:
len(ccle_test.columns)

422

custom dataset for dataloader

In [19]:
seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [20]:


class CCLE_Dataset(Dataset):
    # load the dataset
    def __init__(self, path, train_val_test = 1):
        # load the csv file as a dataframe
        df = path#read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, 0:len(df.columns)-1]
        self.y = df.values[:, len(df.columns)-1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        self.X = StandardScaler().fit_transform(self.X)
        # label encode target and ensure the values are floats
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        x_train, x_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.3, random_state=42)
        #x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, train_size=0.5, random_state=0, stratify = y_test)

        self.train_val_test = train_val_test
        # two modes - train and test
        if( train_val_test ==1):
            self.x_data, self.y_data = x_train, y_train
        else:
            self.x_data, self.y_data = x_test, y_test

    # number of rows in the dataset
    def __len__(self):
        return len(self.x_data)

    # get a row at an index
    def __getitem__(self, idx):
        #print(self.x_data[idx].shape, self.y_data[idx].shape)
        return [self.x_data[idx], self.y_data[idx]]



In [21]:


class mixed_Dataset_for_test(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = path#read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, 0:len(df.columns)-1]
        self.y = df.values[:, len(df.columns)-1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        self.X = StandardScaler().fit_transform(self.X)
        # label encode target and ensure the values are floats
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        #print(self.x_data[idx].shape, self.y_data[idx].shape)
        return [self.X[idx], self.y[idx]]



In [22]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor

# Create linear regression object
# change to RandomForestRegressor() while needed 
#m1 = RandomForestRegressor(max_depth=4)#linear_model.Ridge()
#m2=  RandomForestRegressor(max_depth=4)#linear_model.Ridge()

In [23]:
class Net(nn.Module):
    """Model (simple CNN adapted from 'PyTorch: A 60 Minute Blitz')"""

    def __init__(self) -> None:
        super(Net, self).__init__()
        #self.lin = nn.Linear(620,1)
        self.layers = nn.Sequential(
        nn.Linear(len(CCLE.columns)-1, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layers(x)
        return x

In [24]:
def train(net, trainloader, epochs):
    """Train the model on the training set."""
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    # enumerate epochs
    for epoch in range(epochs):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(trainloader):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = net(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [25]:
def test(net, testloader):
    """Validate the model on the test set."""
    net.eval()
    criterion = torch.nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(testloader):
        # evaluate the model on the test set
        yhat = net(inputs)
        # retrieve numpy array

        #yhat = yhat.detach().numpy()
        #actual = targets.numpy()
        actual = targets.reshape((len(targets), 1))

        loss += criterion(yhat,actual).item()

        #print(yhat)
        predictions.append(yhat.detach().numpy())
        actuals.append(actual.detach().numpy())
    
    #print(type(predictions))
    predictions, actuals= vstack(predictions), vstack(actuals)

    
    print("Mean squared error: %.2f" % mean_squared_error(actuals,predictions))

    #mean absolute error
    print("Mean absolute error: %.2f" % mean_absolute_error(actuals, predictions))

    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(actuals, predictions))

    return mean_squared_error(actuals,predictions), mean_absolute_error(actuals, predictions), r2_score(actuals, predictions)

In [26]:
warnings.filterwarnings("ignore", category=UserWarning)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
def load_data(path):
    # load the dataset
    train = CCLE_Dataset(path,train_val_test=1)
    test =CCLE_Dataset(path,train_val_test=3)
    batch_size =16

    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
    return train_dl, test_dl

In [28]:
m1 = Net().to(DEVICE)
m2= Net().to(DEVICE)

In [29]:
ccle_trainloader, ccle_testloader = load_data(CCLE)
gCSI_trainloader, gCSI_testloader = load_data(gCSI)

In [30]:
# make a class prediction for one row of data
def predict(net, testloader):
    net.eval()
    criterion = torch.nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(testloader):
        # evaluate the model on the test set
        yhat = net(inputs)
        # retrieve numpy array

        #yhat = yhat.detach().numpy()
        #actual = targets.numpy()
        actual = targets.reshape((len(targets), 1))

        loss += criterion(yhat,actual).item()

        #print(yhat)
        predictions.append(yhat.detach().numpy())
        actuals.append(actual.detach().numpy())
    
    #print(type(predictions))
    predictions, actuals= vstack(predictions), vstack(actuals)
    return predictions, actuals


In [31]:
train(m1, ccle_trainloader, epochs=100)
train(m2, gCSI_trainloader, epochs = 100)

In [32]:
#in-study test results ccle 30%
mse_loss_ccle, mae_loss_ccle, r2_ccle = test(m1, ccle_testloader)
print(mse_loss_ccle, mae_loss_ccle, r2_ccle)

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.85
0.010308111 0.07178959 0.8522058936757887


In [33]:
#in-study test results ccle 30%
mse_loss_gCSI, mae_loss_gCSI, r2_gCSI = test(m2, gCSI_testloader)
print(mse_loss_gCSI, mae_loss_gCSI, r2_gCSI)

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.64
0.0099457 0.07287775 0.6404708241268007


In [34]:
#predict(ccle_X_test.iloc[0,:], m1)

In [35]:
#mixed study results trained on CCLE (70%)

mixed_test = pd.concat([ccle_test, gCSI_test], ignore_index=True)

mixed_test = mixed_Dataset_for_test(mixed_test)


In [36]:
mixed_test_dl = DataLoader(mixed_test, batch_size=16, shuffle=False)

In [37]:
#test m1 on mixed test data
mse_loss_mixed_m1, mae_loss_mixed_m1, r2_mixed_m1 = test(m1, mixed_test_dl)
print(mse_loss_mixed_m1, mae_loss_mixed_m1, r2_mixed_m1)

Mean squared error: 0.02
Mean absolute error: 0.10
Coefficient of determination: 0.58
0.020797884 0.09539164 0.5776932576992659


In [38]:
#test m2 on mixed test data
mse_loss_mixed_m2, mae_loss_mixed_m2, r2_mixed_m2 = test(m2, mixed_test_dl)
print(mse_loss_mixed_m2, mae_loss_mixed_m2, r2_mixed_m2)

Mean squared error: 0.02
Mean absolute error: 0.09
Coefficient of determination: 0.61
0.01904553 0.09400541 0.6132753083363416


In [39]:
#m5

mixed_train = pd.concat([ccle_train, gCSI_train], ignore_index=True)

mixed_train = mixed_Dataset_for_test(mixed_train)

mixed_train_dl = DataLoader(mixed_train, batch_size=16, shuffle=True)

m5 = Net().to(DEVICE)

In [40]:
train(m5, mixed_train_dl, epochs=100)
#train(m2, gCSI_trainloader, epochs = 100)

In [41]:
#test m5 on mixed test data
mse_loss_mixed_m5, mae_loss_mixed_m5, r2_mixed_m5 = test(m5, mixed_test_dl)
print(mse_loss_mixed_m5, mae_loss_mixed_m5, r2_mixed_m5)

Mean squared error: 0.02
Mean absolute error: 0.09
Coefficient of determination: 0.69
0.015279466 0.08649029 0.6897462588261971


In [42]:
ccle_y_train_pred =test(m1, ccle_trainloader)
gCSI_y_train_pred = test(m2, gCSI_trainloader)

Mean squared error: 0.01
Mean absolute error: 0.05
Coefficient of determination: 0.93
Mean squared error: 0.00
Mean absolute error: 0.05
Coefficient of determination: 0.86


In [43]:
ccle_y_train_pred, ccle_y_train_m1 = predict(m1, ccle_trainloader)
gCSI_y_train_pred, gCSI_y_train_m2 = predict(m2, gCSI_trainloader)

In [44]:
ccle_y_train_pred

array([[ 1.68653429e-01],
       [ 3.77897769e-02],
       [ 7.53527939e-01],
       [ 8.46741274e-02],
       [ 9.69636217e-02],
       [ 9.79940817e-02],
       [-1.45411789e-02],
       [ 7.75077567e-02],
       [ 3.25964317e-02],
       [ 3.23563993e-01],
       [ 1.69942901e-03],
       [ 4.73017573e-01],
       [-2.04466842e-03],
       [ 7.76223004e-01],
       [ 4.80306819e-02],
       [ 2.71969326e-02],
       [ 8.45858455e-02],
       [ 6.43052533e-02],
       [ 9.44692716e-02],
       [ 4.99899909e-02],
       [ 1.80666223e-02],
       [ 1.27252676e-02],
       [ 4.20735553e-02],
       [ 9.46058109e-02],
       [ 1.64373487e-01],
       [ 6.82834864e-01],
       [ 1.05890103e-01],
       [-6.41055778e-03],
       [ 1.86104536e-01],
       [ 6.45877570e-02],
       [ 1.04904987e-01],
       [ 1.51510298e-01],
       [ 2.97516137e-02],
       [ 6.78163588e-01],
       [ 3.83953989e-01],
       [ 7.86853433e-02],
       [ 4.95260954e-01],
       [ 6.74050078e-02],
       [ 7.5

WA method

In [45]:
ccle_y_pred, ccle_y_test_m1 = predict(m1, ccle_testloader)
gCSI_y_pred, gCSI_y_test_m2 = predict(m2, gCSI_testloader)

In [46]:
avg_y_pred = (ccle_y_pred + gCSI_y_pred)/2

In [47]:
avg_y_pred = pd.Series(avg_y_pred.squeeze())

In [48]:
avg_y_pred_stacked = pd.concat([avg_y_pred, avg_y_pred], ignore_index=True)

In [49]:
mixed_y_test1 = pd.concat([pd.Series(ccle_y_pred.squeeze()),pd.Series(gCSI_y_pred.squeeze())], ignore_index =True)

In [50]:
print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test1, avg_y_pred_stacked))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test1, avg_y_pred_stacked))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test1, avg_y_pred_stacked))

Mean squared error: 0.01
Mean absolute error: 0.05
Coefficient of determination: 0.87


do a linear regression on the training scores

In [51]:
m4 = linear_model.LinearRegression()

In [52]:
m4.fit(gCSI_y_train_pred, ccle_y_train_pred)

# Make predictions using the testing set
ccle_y_pred_m4 = m4.predict(gCSI_y_pred.reshape(-1,1))

#avg ccle values predicted by m4 + gCSI input y values  to m4 
gCSI = (ccle_y_pred_m4 + gCSI_y_pred.reshape(-1,1))/2

#print(gCSI)
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, gCSI))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, gCSI))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, gCSI))

Mean squared error: 0.01
Mean absolute error: 0.10
Coefficient of determination: 0.48


In [53]:
m3 =linear_model.LinearRegression()

In [54]:
m3.fit(ccle_y_train_pred, gCSI_y_train_pred)

# Make predictions using the testing set
gCSI_y_pred_m3 = m3.predict(ccle_y_pred.reshape(-1,1))

ccle = (gCSI_y_pred_m3 + ccle_y_pred.reshape(-1,1))/2
print("Mean squared error: %.2f" % mean_squared_error(ccle_y_test, ccle))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(ccle_y_test, ccle))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(ccle_y_test, ccle))

Mean squared error: 0.02
Mean absolute error: 0.12
Coefficient of determination: 0.64


In [55]:
ccle = pd.Series(ccle.squeeze())
gCSI =pd.Series(gCSI.squeeze())

In [56]:
concated = pd.concat([ccle, gCSI], ignore_index=True)

In [57]:
#test on mixed data 

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test1, concated))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test1, concated))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test1, concated))

Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.74


FL

In [61]:
params1 = []
for param in m1.parameters():
  params1.append(param.data)

params2 = []
for param in m2.parameters():
  params2.append(param.data)

In [64]:
m6 = Net().to(DEVICE)
params3 = iter(params1 + params2)
for param in m6.parameters():
  param.data = next(params3)/2

In [73]:
for param in m6.parameters():
  print(param.data)

tensor([[ 0.0036,  0.0108, -0.0145,  ...,  0.0081,  0.0137, -0.0030],
        [-0.0117,  0.0021,  0.0127,  ...,  0.0078, -0.0052,  0.0132],
        [ 0.0045,  0.0122,  0.0087,  ..., -0.0260, -0.0022,  0.0201],
        ...,
        [ 0.0103,  0.0112,  0.0062,  ...,  0.0018,  0.0179,  0.0172],
        [ 0.0262,  0.0164, -0.0114,  ...,  0.0132,  0.0127,  0.0003],
        [-0.0104,  0.0002, -0.0162,  ..., -0.0183,  0.0016, -0.0114]])
tensor([ 8.9977e-03,  1.9870e-02, -1.6941e-02,  2.1349e-02,  9.1747e-03,
         2.2019e-02,  2.1245e-03, -1.5326e-02, -1.8638e-02,  1.8979e-02,
        -1.4318e-02, -1.5277e-02, -7.7919e-03, -2.3688e-02,  1.2020e-04,
         1.0448e-03,  5.0830e-03, -2.4190e-03, -1.4103e-02,  2.0380e-02,
        -1.1072e-02, -2.0836e-02,  1.9616e-02, -1.9962e-02, -1.8649e-02,
        -2.2164e-02,  1.2700e-02,  6.2640e-03,  1.8169e-02,  1.5722e-02,
        -1.4805e-02,  5.8711e-03,  9.9097e-05,  1.4933e-02, -1.2075e-02,
         2.0918e-02, -6.6211e-03, -1.1992e-02, -2.5365e

In [65]:
#test m6 on mixed test data
mse_loss_mixed_m6, mae_loss_mixed_m6, r2_mixed_m6 = test(m6, mixed_test_dl)
print(mse_loss_mixed_m6, mae_loss_mixed_m6, r2_mixed_m6)

Mean squared error: 0.07
Mean absolute error: 0.17
Coefficient of determination: -0.36
0.06700799 0.16954768 -0.36061563786893003
